# Data Preprocessing with Sklearn

This notebook is going to summarize the key data preprocessing techniques and understanding. `Sklearn` is preprocessing library forms a solid foundation to guide you through this important task in the data science pipeline.

## Outline

* Missing values
* Polynomial features
* Categorical features
* Numerical features
* Custom transformations
* Feature scaling
* Normalization

## 1. Missing values

First of all, most machine learning algorithms can't handle the missing values such as Linear Regression, however, some models have advantages. For example, tree-based models are doing greedy search to minize the impurity; LightGBM has an option `use_missing = false`; SVM's hyperplane is constructed by support vectors.

it is important to identify the missing values and know with which value they are replaced. The simple answer is the decision should partially depend on how random missing values are. If they are completely at random, they don’t give any extra information and can be omitted or fill in. On the other hand, if they’re not at random, the fact that a value is missing is itself information and can be expressed as an extra binary feature.

In [1]:
import numpy as np
import pandas as pd

# Example Missing Data
data = np.array([5,7,8, np.NaN, np.NaN, np.NaN, -5, 0,25,999,1,-1, np.NaN, 0, np.NaN]).reshape((5,3))
data = pd.DataFrame(data, columns = ['f1', 'f2', 'f3']) #feature 1, feature 2, feature 3
data

,f1,f2,f3
0,5.0,7.0,8.0
1,NaN,NaN,NaN
2,-5.0,0.0,25.0
3,999.0,1.0,-1.0
4,NaN,0.0,NaN


In [5]:
# check row-wise missing value 
data[data.isna().any(axis=1)]

,f1,f2,f3
1,NaN,NaN,NaN
4,NaN,0.0,NaN


### 1.1. Drop NA

Rows or columns with to many non-meaningful missing values can be deleted from you data with `dropna` method. However, it will delete row number at the same time, so `reset_index` will be a good idea

* axis: 0 for rows, 1 for columns
* thresh: the number of non-NaN’s exists so that keep the row or column 
* inplace: update the frame

In [9]:
# drop all NA
data.dropna()

,f1,f2,f3
0,5.0,7.0,8.0
2,-5.0,0.0,25.0
3,999.0,1.0,-1.0


In [10]:
# drop 2+ NA
data.dropna(thresh=1)

,f1,f2,f3
0,5.0,7.0,8.0
2,-5.0,0.0,25.0
3,999.0,1.0,-1.0
4,NaN,0.0,NaN


In [11]:
# drop NA, reset indexing
data.dropna().reset_index(drop=True)

,f1,f2,f3
0,5.0,7.0,8.0
1,-5.0,0.0,25.0
2,999.0,1.0,-1.0


### 1.2.1. Imputing NA - Stats

For filling up missing values with common strategies, the pandas provides replace and fillna method to impute missing value. The four main strategies are mean, most_frequent, median and constant for continuous variables, or most frequent value for categorical variables

* easy, quick, good for small size dataset
* mean changes data distribution, causing overfiting
* only column-wise calculation, missing correlation between features
* bad performance on encoded categorical variables

In [13]:
# fillna by mean
data.fillna(data.mean())

,f1,f2,f3
0,5.0,7.0,8.000000
1,333.0,2.0,10.666667
2,-5.0,0.0,25.000000
3,999.0,1.0,-1.000000
4,333.0,0.0,10.666667


In [15]:
from sklearn.impute import SimpleImputer

# mean, median, most_frequent
imp_mean = SimpleImputer(strategy='mean')
imp_mean.fit_transform(data)

array([[  5.        ,   7.        ,   8.        ],
       [333.        ,   2.        ,  10.66666667],
       [ -5.        ,   0.        ,  25.        ],
       [999.        ,   1.        ,  -1.        ],
       [333.        ,   0.        ,  10.66666667]])

This pandas implementation also provides options to fill forward (ffill) or fill backward (bfill), which are convenient when working with time series.

In [12]:
# replace NA with 999 and 0 
data.replace([999.0,0], np.NaN)

,f1,f2,f3
0,5.0,7.0,8.0
1,NaN,NaN,NaN
2,-5.0,NaN,25.0
3,NaN,1.0,-1.0
4,NaN,NaN,NaN


### 1.2.2. Imputing NA - Modeling

Other popular ways to impute missing data using the k-nearest neighbor (KNN) algorithm to predict the missing value. The filled value is calculated by k-nearest data points that their distribution is similiar as missing data.

* better accuracy
* consider multi-dimensional feature
* go through entire training set, memory issue, heavy computation
* sensitive with outliers

In [18]:
from impyute.imputation.cs import fast_knn

imputed_training = fast_knn(data, k=2)
imputed_training

,0,1,2
0,5.000000,7.000000,8.000000
1,6.987584,6.957582,8.016159
2,-5.000000,0.000000,25.000000
3,999.000000,1.000000,-1.000000
4,6.987364,0.000000,8.016157


MICE Algorithm

* fexible
* handle continuous, binary, categorical data

In [20]:
from impyute.imputation.cs import mice

imputed_training = mice(data)
imputed_training

,0,1,2
0,5.000000,7.0,8.000000
1,400.911111,2.0,10.666667
2,-5.000000,0.0,25.000000
3,999.000000,1.0,-1.000000
4,604.644444,0.0,10.666667


Stochastic regression imputation

Regression imputation is training a regression model using related features to predict missing value, and stochastic regression imputation is adding aditional random residual term on above algorithm. 

### 2. Polynomial features

**Creating polynomial features is a simple and common way of feature engineering that adds complexity to numeric input data by combining features.** Polynomial features are often created when we want to include the notion that there exists a nonlinear relationship between the features and the target. They are mostly used to add complexity to linear models with little features, or when we suspect the effect of one feature is dependent on another feature.

If you for example replace all the missing values by 0, all the cross-products using this feature will be 0. Moreover, if you don’t replace missing values (NaN), creating polynomial features will raise a value error.

Sklearn provides a `PolynomialFeatures` class to create polynomial features from scratch. The degree parameter determines the maximum degree of the polynomial. For example, when degree is set to two, the features created will be 1, $x_1$, $x_2$, $x^2$ and $x_1x_2$. The interaction_only parameter let the function know we only want the interaction features, i.e. 1, $x_1$, $x_2$, and $x_1x_2$.

In [14]:
from sklearn.preprocessing import PolynomialFeatures

# Replace 999 and NaN to mean
data_clean = data.replace(999.0, np.NaN).fillna(data.mean())
print(data_clean)

poly = PolynomialFeatures(degree=3, interaction_only=True) # only create interaction terms

polynomials = pd.DataFrame(poly.fit_transform(data_clean))
polynomials

      f1   f2         f3
0    5.0  7.0   8.000000
1  333.0  2.0  10.666667
2   -5.0  0.0  25.000000
3  333.0  1.0  -1.000000
4  333.0  0.0  10.666667


,0,1,2,3,4,5,6,7
0,1.0,5.0,7.0,8.000000,35.0,40.0,56.000000,280.0
1,1.0,333.0,2.0,10.666667,666.0,3552.0,21.333333,7104.0
2,1.0,-5.0,0.0,25.000000,-0.0,-125.0,0.000000,-0.0
3,1.0,333.0,1.0,-1.000000,333.0,-333.0,-1.000000,-333.0
4,1.0,333.0,0.0,10.666667,0.0,3552.0,0.000000,0.0


Just as with any other form of feature engineering, it is important to create polynomial features before doing any feature scaling.

## 3. Categorical features

The dataset will contain categorical variables. These variables are typically stored as text values which represent various traits. Some examples include color (`ordinal`) and size (`nominal`). Many machine learning algorithms can support categorical values without further manipulation but there are many more algorithms that do not, so it's necessary to convert categorical features to a numerical representation.

Before you start transforming your data, it is important to figure out if the feature you’re working on is ordinal (as opposed to nominal). An ordinal feature is best described as a feature with natural, ordered categories and the distances between the categories is not known.

In [22]:
x = np.array(['M', 'O', 'medium', 'M', 'O', 'high','F', 'A', 'high', 'F', 'AB', 'low','F', 'B', np.NaN]).reshape(5,3)

data = pd.DataFrame(x, columns = ['sex', 'blood_type', 'edu_level'])
data

,sex,blood_type,edu_level
0,M,O,medium
1,M,O,high
2,F,A,high
3,F,AB,low
4,F,B,nan


### 3.1. One Hot Encoding

**The most popular way to encode categorical features is one-hot-encoding. The basic strategy is to convert each category value into a new column and assigns a 1 or 0 (True/False) value to the column.** This has the benefit of not weighting a value improperly but does have the downside of adding more columns to the data set. Essentially, each categorical feature with n categories is transformed into n binary features.

* Curse of Dimensionality when there are many categories, causing model overfiting
* Sparsity loses features power, continuous variables higher feature importance
* Dense Representation: heavy memory
* Sparse Representation: efficient, but require models support sparse matrics (xgboost, LGBM)

In [23]:
# prefix params rename column
pd.get_dummies(data)

,sex_F,sex_M,blood_type_A,blood_type_AB,blood_type_B,blood_type_O,edu_level_high,edu_level_low,edu_level_medium,edu_level_nan
0,0,1,0,0,0,1,0,0,1,0
1,0,1,0,0,0,1,1,0,0,0
2,1,0,1,0,0,0,1,0,0,0
3,1,0,0,1,0,0,0,1,0,0
4,1,0,0,0,1,0,0,0,0,1


In [48]:
from sklearn.preprocessing import OneHotEncoder

# categories: unique value per feature, set as training set feature, or input customerized feature list
# sparse: return matrix if True, else return array
# dtype: set as np.int
# handle_unknown: set ignore for missing feature 

enc = OneHotEncoder(dtype=np.int, sparse=True, handle_unknown='ignore')
pd.DataFrame(enc.fit_transform(data).toarray(), columns=pd.get_dummies(data).columns) # enc.get_feature_names()

,sex_F,sex_M,blood_type_A,blood_type_AB,blood_type_B,blood_type_O,edu_level_high,edu_level_low,edu_level_medium,edu_level_nan
0,0,1,0,0,0,1,0,0,1,0
1,0,1,0,0,0,1,1,0,0,0
2,1,0,1,0,0,0,1,0,0,0
3,1,0,0,1,0,0,0,1,0,0
4,1,0,0,0,1,0,0,0,0,1


### 3.2. Numeric Encoding

Another approach to encoding categorical values is to use a technique called numberic encoding. Numeric encoding is simply converting each value in a column to a number.

Pandas provides `Categorical` and `factorize` methods to tranform both ordinal and nominal categorical type variables that also can handle missing values and respects the order of our values. For unknown category, we can create a new level, or replace by the most common category.

In `Sklearn` library, **`OrdinalEncoder` is for converting features, while `LabelEncoder` is for converting target variable.** OrdinalEncoder can fit data that has the shape of (n_samples, n_features) while LabelEncoder can only fit data that has the shape of (n_samples,). **However, both methods do not support unknown category, so it might cause an issue when deploy models.**

* Intuitive, straight-forward and may give you a good performance
* Numerical values can be misinterpreted by the algorithm

In [46]:
cat = pd.Categorical(data.edu_level, categories=['missing', 'low', 'medium', 'high'], ordered=True)
cat = cat.fillna('missing')
labels, unique = pd.factorize(cat, sort=True)

pd.DataFrame([cat, labels], columns=['edu_level', 'Encoding'])

,edu_level,Encoding
0,medium,2
1,high,3
2,high,3
3,low,1
4,missing,0


In [26]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
pd.DataFrame(enc.fit_transform(data), columns = data.columns)

,sex,blood_type,edu_level
0,1.0,3.0,2.0
1,1.0,3.0,0.0
2,0.0,0.0,0.0
3,0.0,1.0,1.0
4,0.0,2.0,3.0


In [53]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
pd.DataFrame(enc.fit_transform(data.edu_level), columns=[data.edu_level.name])

,edu_level
0,2
1,0
2,0
3,1
4,3


### 3.3. Binary Encoding

Depending on the data set, you may be able to use binary encoding to hash the cardinalities into binary values. It's a combination of label encoding and one hot encoding to create a binary column that meets your needs for further analysis. In this technique, first the categories are encoded as ordinal, then those integers are converted into binary code, then the digits from that binary string are split into separate columns. This encodes the data in fewer dimensions than one-hot.

In [55]:
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['sex', 'blood_type', 'edu_level'])
df_binary = encoder.fit_transform(data)

df_binary

,sex_0,sex_1,blood_type_0,blood_type_1,blood_type_2,edu_level_0,edu_level_1,edu_level_2
0,0,1,0,0,1,0,0,1
1,0,1,0,0,1,0,1,0
2,1,0,0,1,0,0,1,0
3,1,0,0,1,1,0,1,1
4,1,0,1,0,0,1,0,0


### 3.4. Miscellaneous Features

You may encounter categorical feature columns which specify the ranges of values for observation points, for example, the age column might be described in the form of categories like 0-20, 20-40. The most common ones are either split these ranges into two separate columns or replace them with some measure like the mean of that range.

In [70]:
df = pd.DataFrame({'age':['0-20', '20-40', '40-60','60-80']})
df[['start', 'end']] = pd.DataFrame(df.age.str.split('-').to_list(), index=df.index)
df

,age,start,end
0,0-20,0,20
1,20-40,20,40
2,40-60,40,60
3,60-80,60,80


## 4. Numerical features

Just like categorical data can be encoded, numerical features can be decoded into categorical features. The two most common ways to do this are **discretization** and **binarization**.

### 4.1. Discretization

Discretization divides a continuous feature into a pre-specified number of categories. **One of the main goals of a discretization is to significantly reduce the number of discrete intervals of a continuous attribute. Hence, why this transformation can increase the performance of tree based models.**

Sklearn provides a `KBinsDiscretizer` class that can take care of this. The only thing you have to specify are the number of bins (n_bins) for each feature and how to encode these bins (ordinal, onehot or onehot-dense).

The optional strategy parameter can be set to three values:

* uniform: all bins in each feature have identical widths,but it is very sensitive for outliers
* quantile (default): all bins in each feature have the same number of points.
* kmeans: all values in each bin have the same nearest center of a 1D k-means cluster.

In [71]:
from sklearn.preprocessing import KBinsDiscretizer

x = [[-2, 1, -4,   -1],
     [-1, 2, -3, -0.5],
     [ 0, 3, -2,  0.5],
     [ 1, 4, -1,    2]]
est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
est.fit_transform(x)

array([[0., 0., 0., 0.],
       [1., 1., 1., 0.],
       [2., 2., 2., 1.],
       [2., 2., 2., 2.]])

### 4.2. Binarization

Feature binarization is the process of tresholding numerical features to get boolean values. Or in other words, assign a boolean value (True or False) to each sample based on a threshold.

In general binarization is useful as a feature engineering technique for creating new features that indicate something meaningful. The `Binarizer` class in sklearn implements binarization in a very intuitive way. The only parameters you need to specify are the threshold and copy. All values below or equal to the threshold are replaced by 0, above it by 1. 

In [13]:
from sklearn.preprocessing import Binarizer

x = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

# binary threshold 0
transformer = Binarizer(threshold = 0)
transformer.fit_transform(x)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

## 5. Feature Scaling - Standardization

Before applying any scaling transformations, it is very important to **split your data into a train set and a test set**. If you start to scale data before, then your model already looked up the information from test data, which it makes test data non-generalized to the real world and causes overfitting. Therefore, we don't want test data to be a part of data processing or model training step.

**Many rescaling methods (standardization and min-max scaling) operate on features however, we can also rescale (normalization) across individual observations**.

### 5.1. Standardization

Standardization is a transformation that **centers the data by removing the mean value of each feature, and then scale it by dividing (non-constant) features by their standard deviation.** After standardizing data the mean will be zero and the standard deviation one.

Standardization can drastically improve the performance of models. For instance, many elements used in the objective function of a learning algorithm (such as the RBF kernel of Support Vector Machines or the $L1$ and $L2$ regularizers of linear models) assume that all features are centered around zero and have variance in the same order. **If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.**

For example, principal component analysis often works better using standardization, while min-max scaling is often recommended for neural networks.

**Standard Scaler**: It purely centers the data by using the following formula, where $\mu$ is the mean and $\sigma$ is the standard deviation. Remember that the value of our fourth instance was missing, and we replaced it by the mean.

std_scaler = $\frac{x-\mu}{\sigma}$

In [14]:
from sklearn.preprocessing import StandardScaler

print('Original Data')
print(data, '\n')

print('Filled Missing value')
print(data.fillna(data.mean()))

scaler = StandardScaler()
scaler.fit_transform(data.fillna(data.mean()))

Original Data
      f1   f2    f3
0    5.0  7.0   8.0
1    NaN  NaN   NaN
2   -5.0  0.0  25.0
3  999.0  1.0  -1.0
4    NaN  0.0   NaN 

Filled Missing value
      f1   f2         f3
0    5.0  7.0   8.000000
1  333.0  2.0  10.666667
2   -5.0  0.0  25.000000
3  999.0  1.0  -1.000000
4  333.0  0.0  10.666667


array([[-0.89913037,  1.91741247, -0.31933647],
       [ 0.        ,  0.        ,  0.        ],
       [-0.92654289, -0.76696499,  1.71643352],
       [ 1.82567326, -0.38348249, -1.39709705],
       [ 0.        , -0.76696499,  0.        ]])

### 5.2. MinMax Scaler

The MinMaxScaler transforms features by scaling each feature to a given range. This range can be set by specifying the feature_range parameter. **This scaler works better for cases where the distribution is not Gaussian or the standard deviation is very small**. However, it is **sensitive to outliers**, so if there are outliers in the data, you might want to consider another scaler.

MinMax_scaler = $\frac{x-min(x)}{max(x) - min(x)}$

In [15]:
from sklearn.preprocessing import MinMaxScaler

print(data.fillna(data.mean()))
scaler = MinMaxScaler(feature_range=(-10,10))
scaler.fit_transform(data.fillna(data.mean()))

      f1   f2         f3
0    5.0  7.0   8.000000
1  333.0  2.0  10.666667
2   -5.0  0.0  25.000000
3  999.0  1.0  -1.000000
4  333.0  0.0  10.666667


array([[ -9.80079681,  10.        ,  -3.07692308],
       [ -3.26693227,  -4.28571429,  -1.02564103],
       [-10.        , -10.        ,  10.        ],
       [ 10.        ,  -7.14285714, -10.        ],
       [ -3.26693227, -10.        ,  -1.02564103]])

### 5.3. MaxAbs Scaler

The MaxAbsScaler works very similarly to the MinMaxScaler but automatically scales the data to a [-1, 1] range based on the **absolute maximum**. This scaler is meant for **data that is already centered at zero or sparse data**. It does not shift/center the data, and thus does not destroy any sparsity.

MaxAbs_scaler = $\frac{x}{Max|x|}$

In [16]:
from sklearn.preprocessing import MaxAbsScaler

print(data.fillna(data.mean()))
scaler = MaxAbsScaler()
scaler.fit_transform(data.fillna(data.mean()))

      f1   f2         f3
0    5.0  7.0   8.000000
1  333.0  2.0  10.666667
2   -5.0  0.0  25.000000
3  999.0  1.0  -1.000000
4  333.0  0.0  10.666667


array([[ 0.00500501,  1.        ,  0.32      ],
       [ 0.33333333,  0.28571429,  0.42666667],
       [-0.00500501,  0.        ,  1.        ],
       [ 1.        ,  0.14285714, -0.04      ],
       [ 0.33333333,  0.        ,  0.42666667]])

### 5.4. Robust Scaler

If your data contains many outliers, scaling using the mean and standard deviation of the data is likely to not work very well. In these cases, you can use the `RobustScaler`. **It removes the median and scales the data according to the quantile range.**

By default, the scaler uses the Inter Quartile Range (IQR), which is the range between the 1st quartile and the 3rd quartile. The quantile range can be manually set by specifying the quantile_range parameter when initiating a new instance of the `RobustScaler`.

In [17]:
from sklearn.preprocessing import RobustScaler

print(data.fillna(data.mean()))
robust = RobustScaler(quantile_range = (0.1,0.9))
robust.fit_transform(data.fillna(data.mean()))

      f1   f2         f3
0    5.0  7.0   8.000000
1  333.0  2.0  10.666667
2   -5.0  0.0  25.000000
3  999.0  1.0  -1.000000
4  333.0  0.0  10.666667


array([[-1.02500000e+03,  6.00000000e+00, -9.25925926e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.05625000e+03, -1.00000000e+00,  4.97685185e+01],
       [ 2.08125000e+03,  0.00000000e+00, -4.05092593e+01],
       [ 0.00000000e+00, -1.00000000e+00,  0.00000000e+00]])

## 6. Feature scaling - Normalization

Normalization is the process of the **values on individual observations** to have unit norm (the sum of their lengths is 1). This type of rescaling is often used when we have many equivalent features (e.g., text classification when every word or n-word group is a feature).

In basic terms you need to normalize data when the algorithm predicts based on the weighted relationships formed between data points. Scaling inputs to unit norms is a common operation for text classification or clustering.

Although there are many other ways to normalize data, sklearn provides three norms: **L1, L2 and Max**. When creating a new instance of the Normalizer class you can specify the desired norm under the norm parameter.

### 6.1. Max Normalize

The max norm uses the absolute maximum and does for samples what the MaxAbsScaler does for features.

Max_normalizer = $\frac{x}{Max(x)}$

In [18]:
x = data.fillna(data.mean())
print(x)

norm_max = list(max(list(abs(i) for i in x.iloc[r])) for r in range(len(x)))

max_norm_data = pd.DataFrame(np.zeros((5,3)), columns=['f1','f2','f3'])
for i in range(len(x)):
    max_norm_data.iloc[i,] = x.iloc[i]/norm_max[i]

max_norm_data

      f1   f2         f3
0    5.0  7.0   8.000000
1  333.0  2.0  10.666667
2   -5.0  0.0  25.000000
3  999.0  1.0  -1.000000
4  333.0  0.0  10.666667


,f1,f2,f3
0,0.625,0.875000,1.000000
1,1.000,0.006006,0.032032
2,-0.200,0.000000,1.000000
3,1.000,0.001001,-0.001001
4,1.000,0.000000,0.032032


### 6.2. L1 Norm

Using the sum of all the values as and thus gives equal penalty to all parameters, enforcing sparsity.

L1_norm = $\frac{x}{sum(x)}$

In [19]:
norm_l1 = list(sum(list(abs(i) for i in x.iloc[r])) for r in range(len(x)))

norm_l1_data = pd.DataFrame(np.zeros((5,3)), columns=['f1','f2','f3'])
for i in range(len(x)):
    norm_l1_data.iloc[i,] = x.iloc[i]/norm_l1[i]
    
norm_l1_data

,f1,f2,f3
0,0.250000,0.350000,0.400000
1,0.963356,0.005786,0.030858
2,-0.166667,0.000000,0.833333
3,0.998002,0.000999,-0.000999
4,0.968962,0.000000,0.031038


### 6.3. L2 Norm

Using the square root of the sum of all the squared values. This creates smoothness and rotational invariance. Some models, like PCA, assume rotational invariance, and so l2 will perform better.

L2_norm = $\frac{x}{\sqrt{\sum{x_i^2}}}$

In [20]:
import math
norm_l2 = list(math.sqrt(sum(list((i**2) for i in x.iloc[r]))) for r in range(len(x)))

norm_l2_data = pd.DataFrame(np.zeros((5,3)), columns=['f1','f2','f3'])
for i in range(len(x)):
    norm_l2_data.iloc[i,] = x.iloc[i]/norm_l2[i]
    
norm_l2_data

,f1,f2,f3
0,0.425628,0.595880,0.681005
1,0.999469,0.006003,0.032015
2,-0.196116,0.000000,0.980581
3,0.999999,0.001001,-0.001001
4,0.999487,0.000000,0.032016
